In [1]:
import pandas as pd
import numpy as np

In [2]:
client_df = pd.read_excel("./Sitelist.xlsx")

In [3]:
impressions_df = pd.read_csv("./impressions.csv")

In [4]:
impressions_df = impressions_df[impressions_df.hostname.isin(list(client_df.domain.unique()))]

In [5]:
import json

def extract_pid(placement_id):
    try:
        placement_id = json.loads(placement_id)
    except:
        placement_id = {"tag_id": [""], "bam_ad_slot": []}
    bam_ad_slots = placement_id["bam_ad_slot"]
    # print(bam_ad_slots)
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        # print('.')
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

#impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
#impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)

#impressions_df = impressions_df[impressions_df["pid_type"] == 'int']
#impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

In [6]:
grouped_df = impressions_df.groupby(["hostname"]).apply(
    lambda x: pd.Series(
        {
            "ImpressionCount": x["id"].count(),
            "FixationCount": x.loc[x["is_fixated"], "id"].count()
        }
    )
)
grouped_df = grouped_df.reset_index()
grouped_df

,hostname,ImpressionCount,FixationCount
0,56kilo.se,1069,18
1,aftenposten.no,348,23
2,aftonbladet.se,1697532,153725
3,aktivtraning.se,847,4
4,aktuellhallbarhet.se,1726,13
...,...,...,...
296,vt.se,32,0
297,vxonews.se,183,9
298,webbkameror.se,689,2
299,ystadsallehanda.se,132,8


In [8]:
missing_hostnames = {x for x in client_df.domain if x not in list(grouped_df.hostname)}
missing_hostnames

{'apps.apple.com/us/app/aftonbladet-nyheter/id284962281?uo=4',
 'apps.apple.com/us/app/blocket-k√∂p-s√§lj-begagnat/id323710525?uo=4',
 'apps.apple.com/us/app/hitta-se/id284963764?uo=4',
 'apps.apple.com/us/app/omni-nyheter/id719533376?uo=4',
 'apps.apple.com/us/app/sportbladet/id429681201?uo=4',
 'apps.apple.com/us/app/svenska-dagbladet/id423779856?uo=4',
 'apps.apple.com/us/app/tv-nu-streaming-tv-guide/id284963557?uo=4',
 'bt.dk/fodbold',
 'bt.dk/kendte',
 'bt.dk/krimi',
 'bt.dk/musik',
 'bt.dk/royale',
 'bt.dk/samfund',
 'bt.dk/sport',
 'bt.dk/superligaen',
 'bt.dk/tennis',
 'bt.dk/udland',
 'delfi.lt',
 'dnevno.hr',
 'ekstrabladet.dk/musik',
 'ekstrabladet.dk/nyheder/krigogkatastrofer',
 'ekstrabladet.dk/nyheder/politik',
 'ekstrabladet.dk/sport',
 'elsasentourage.se',
 'fagersta-posten.se',
 'forni.se',
 'fryksdalsbygden.se',
 'harrydaposten.se',
 'ifokus.se',
 'itunes.apple.com/us/app/klart-se-v√§derapp/id352024322',
 'kt-kuriren.se',
 'lrytas.lt',
 'minboll.se',
 'msn.com/pl-pl',

In [11]:
grouped_df.to_excel("./results/groupm_longtail.xlsx", index=False)